In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
base = pd.read_csv('../data/raw/desafio_indicium_imdb.csv')

### PRIMEIRAS ANÁLISES E TRATAMENTO DOS DADOS

In [ ]:
base.info()

Há alguns valores nulos nas colunas que podem ser tratados ou em alguns casos excluidos(podendo excluir até a coluna por completo) dependendo da sua importancia para a analise. 

É possivel notar também que alguns dados estão com tipos inapropriados e para melhorar a analise é necessario fazer a conversão.

In [ ]:
base.describe()

Esses são os dados estatisticos apenas das colunas númericas. Observo que o desvio padrão da coluna IMDB_Rating é baixissimo, concluindo-se a baixa diversidade de notas. 

A média e a mediana das colunas IMDB_Rating e Meta_score estão equilibradas, o que significa que para essas colunas, não há discrepancias nos seus valores.

In [ ]:
base.duplicated().sum() #Não há dados duplicados.

In [ ]:
base.drop('Unnamed: 0', axis=1,inplace=True) #Eliminando uma coluna desnecessária

In [ ]:
base = base[base['Released_Year'] != 'PG']
base['Released_Year'] = base['Released_Year'].astype(int)

Achei importante converter a coluna do ano de lançamento que estava em string para um inteiro (Não converti para datetime por que tem apenas o ano e não a data completa). Porém ao tentar fazer a conversão me deparei com um erro que indicava uma linha inválida que apresentava as letras PG (imcompatível com a conversão).

Então retornei a base de dados para ela mesma, excluindo essa linha e só depois foi possivel fazer a conversão.

In [ ]:
base['Gross'] = base['Gross'].str.replace(r'[^0-9]','',regex=True)
base['Gross'] = base['Gross'].astype(float)


Para a coluna Gross, que é o faturamento fiz a conversão para o tipo float (por se tratar de uma coluna com valores financeiros). 

Para isso, antes foi necessário substituir todos os caracteres não numericos por strings vazias e só depois fazer a conversão. 



In [ ]:
base.describe()

Ao converter a coluna de Gross, posso perceber que não há um equilibrio entre a media e mediana indicando números discrepantes. 

Decidi manter as linhas das colunas que possuem valores nulos e preenche-los com a  mediana para garantir que o preencimento seja equilibrado. Fazendo isso também na coluna Meta_score

In [ ]:
base['Gross'] = base['Gross'].fillna(base['Gross'].median())
base['Meta_score'] = base['Meta_score'].fillna(base['Meta_score'].median())             

A coluna Runtime apresenta o tempo do filme, porém em formato de string. Para análises estatísticas e treinamento de machine learning é imprtante que a conversão para um tipo numerico seja feita.

In [ ]:
base['Runtime'] = base['Runtime'].str.replace(r'[^0-9]','', regex=True) #Eliminando caracteres não númericos
base['Runtime'] = base['Runtime'].astype(int)
base.rename(columns={'Runtime':'Runtime_Min'},inplace=True) #Julguei necessário renmear a coluna para que ela ficasse mais clara após a conversão dos seus dados 

Tratando a coluna Certificate com as legendas adequadas para a classificação etária e preechendo os valores nulos.

Segui principalmente a legenda sugerida pelo site: Motion Picture Association

In [ ]:
remover = ['Approved','TV-PG','GP','TV-14','Unrated','Passed','TV-MA','16']
base = base[~base['Certificate'].isin(remover)]

legenda = {'A':'Adult','R':'Adult','U':'All Ages','G':'All Ages','U/A':'Parental Guidance',
          'UA':'Parental Guidance','PG':'Parental Guidance','PG-13':'Parents Strongly Cautioned'}

base['Certificate'] = base['Certificate'].fillna('Unknown')
           
base['Certificate'] = base['Certificate'].replace(legenda)

### INSIGHTS

In [ ]:
correlacao = base[['Released_Year','Runtime_Min','IMDB_Rating','Meta_score','No_of_Votes','Gross']].corr()


In [ ]:
sns.heatmap(correlacao,annot=True,cmap="magma",fmt=".2f")
plt.show()


In [ ]:
plt.figure(figsize=(12,5))
plt.bar(base['Released_Year'],base['Runtime_Min'])
plt.xlabel('Ano de Lançamento')
plt.ylabel('Duração em Minutos')
plt.title('Evolução de duração dos filmes | 1920 - 2020')
plt.grid(True)
plt.show()

Interessante notar essa correlação entre a duração do filme e o ano de lançamento. A tendência aqui indica um aumento de duração, porém com dados que vão até 2020. 

Será que em 2025 e anos por vir essa tendencia continuaria? Podemos observar o contexto e a sociedade atualmente, mas o que será que os dados dizem? 

Será que filmes com maiores durações são populares ou atraem um público diverso, visto que hoje em dia há uma 'urgência' para consumir conteúdo de forma rápida? Se sim, quais fatores atraem esse público?

In [ ]:
plt.figure(figsize=(12,5))
sns.scatterplot(x='No_of_Votes',y='Gross',data=base)
plt.xlabel('Número de votos IMDB')
plt.ylabel('Faturamento')
plt.title('Faturamento vs. Popularidade | Votos IMDb')
plt.grid(True)
plt.show()

Alta correlação entre o número de votos no IMDb e o faturamento do filme, o que faz sentido já que o voto são das pessoas que o assistiu. 

Quanto mais pessoas assitirem, maior será a quantidade de pessoas votantes e maior será o seu faturamento. 

Porém há uma outra questão, existem filmes que foram lançados antes da criação do IMDb, que podem até ter votos atualmente, mas não deve ser a única métrica de popularidade já que só a nota pode não expressar o sentimento causado ou a popularidade do filme na epoca em que foi lançado. 

In [ ]:
indicar_filmes = base.loc[(base['IMDB_Rating'] >= 8.5) & (base['Meta_score'] >= 85)]
indicar_filmes.head(7)

 
1ª Indicação - The Godfather (1972) que é um filme atemporal com uma nota 9.2 no IMDb, que apesar de não ser o único indicativo de popularidade é uma boa refêrencia. Segundo o próprio IMDb filmes com 9.0 pontos ou mais, valem a pena ser vistos. Além disso ele também tem nota máxima da critica, reforçando a sua qualidade.

2ª Indicação - The Lord of the Rings: The Fellowship of the Ring (2001) que possui boas notas, mas que também faz parte de generos populares e é aberto a todas as idades. Caso a pessoa tenha uma familia por exemplo, poderá agradar a todos com a escolha.


In [ ]:
alto_faturamento = base.loc[(base['Gross'] >= 700000000)]
alto_faturamento

In [ ]:
baixo_faturamento = base.loc[(base['Gross'] <= 10000000)]
baixo_faturamento.head(7)


Ao observar os filmes com alto e baixo faturamento, pude perceber que as variaveis que estão associadas as expectativas de faturamento são: diretores, se será uma grande produção (por exemplo as produções de hollywood), o elenco, o idioma, bem como o gênero do filme (os mais populares são: ação,aventura,sci-fi).